In [ ]:
from ollama import Client, chat
import pandas as pd
import numpy as np
import ast
import json
from ollama import ChatResponse
from pydantic import BaseModel
from typing import List



In [ ]:
sem_eval_root = 'SemEval_Task7'
sem_eval_test_root = 'SemEval_Task7_Test_Phase'

In [ ]:
from ollama import Client
# Setup Ollama Client
client = Client(
  host='http://localhost:8081'
)

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb.utils.batch_utils import create_batches

# Setup Chromadb and Embedding Function

ef = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2", device="cuda")

chroma_client = chromadb.PersistentClient()

collection = chroma_client.get_or_create_collection(name="fact_check_collection", embedding_function=ef)

In [ ]:
# Read the test dataset for inference
test_fact_checks_df = pd.read_csv(f'{sem_eval_test_root}/fact_checks.csv')
test_posts_df = pd.read_csv(f'{sem_eval_test_root}/posts.csv')


with open(f'{sem_eval_test_root}/crosslingual_predictions.json', 'r') as crosslingual_file:
  crosslingual_data = json.load(crosslingual_file)

crosslingual_prediction_post_id = list(map(lambda x: int(x), crosslingual_data))

In [ ]:
# Get translated english versions of text
def get_translated_claim(tuple_string):
  return ast.literal_eval(tuple_string)[1]

def get_translated_claim_lang(tuple_string):
  if isinstance(ast.literal_eval(tuple_string)[-1][0], str):
    return ast.literal_eval(tuple_string)[-1][0]
  else:
    return ast.literal_eval(tuple_string)[-1][0][0]

def get_translated_title(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  return ast.literal_eval(tuple_string)[1]


def get_translated_title_lang(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  if isinstance(ast.literal_eval(tuple_string)[-1][0], str):
    return ast.literal_eval(tuple_string)[-1][0]
  else:
    return ast.literal_eval(tuple_string)[-1][0][0]

def get_translated_ocr(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[0][1]

def get_translated_ocr_lang(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[0][-1][0][0]

def get_translated_text(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[1]

def get_translated_text_lang(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[-1][0][0]

In [ ]:
# Data preprocessing for testing phase fact checks
claim_translated = test_fact_checks_df['claim'].apply(get_translated_claim)
claim_lang = test_fact_checks_df['claim'].apply(get_translated_claim_lang)


test_fact_checks_df['claim_translated'] = claim_translated
test_fact_checks_df['claim_lang'] = claim_lang


title_translated = test_fact_checks_df['title'].apply(get_translated_title)
title_lang = test_fact_checks_df['title'].apply(get_translated_title_lang)

test_fact_checks_df['title_translated'] = title_translated
test_fact_checks_df['title_lang'] = title_lang

final_test_fact_checks_df = test_fact_checks_df.drop(columns=['claim', 'title', 'instances'])
final_test_fact_checks_df.tail()

In [ ]:
# Data preprocessing for testing phase social media posts
ocr_translated = test_posts_df['ocr'].apply(get_translated_ocr)
ocr_lang = test_posts_df['ocr'].apply(get_translated_ocr_lang)
test_posts_df["ocr_translated"]  = ocr_translated
test_posts_df["ocr_lang"]  = ocr_lang

text_translated = test_posts_df['text'].apply(get_translated_text)
text_lang = test_posts_df['text'].apply(get_translated_text_lang)
test_posts_df["text_translated"]  = text_translated
test_posts_df["text_lang"]  = text_lang

final_test_posts_df = test_posts_df.drop(columns=["instances", "ocr", "verdicts", "text"])
final_test_posts_df.tail()

In [ ]:
# Upsert fact checks into chromadb
fact_check_id_chroma = final_test_fact_checks_df["fact_check_id"].apply(lambda x:str(x)).to_list()
claims_chroma = final_test_fact_checks_df["claim_translated"].to_list()


batches = create_batches(api=chroma_client,ids=fact_check_id_chroma, documents=claims_chroma)

cur = 0

for batch in batches:
    print(f"Adding batch of size {len(batch[0])}")
    collection.add(ids=batch[0],
                   documents=batch[3])

In [ ]:
# Pydantic class for ollama structured model output
class ClaimGeneratorResponse(BaseModel):
    claim: str

In [ ]:
# Claim Extraction Using deepseek-r1:14b
post_id_generated_claim = {}

i = 0
for post_id in crosslingual_prediction_post_id:
  ocr = final_test_posts_df[final_test_posts_df['post_id'] == post_id]["ocr_translated"].iloc[0]
  if pd.isna(ocr):
    ocr = ""

  text = final_test_posts_df[final_test_posts_df['post_id'] == post_id]["text_translated"].iloc[0]
  if pd.isna(text):
    text = ""

    

  cur_claim_prompt  = f"""
Task: Generate a concise and accurate claim made by a social media post.

Input:

    OCR Text: {ocr}
    Social Media Caption: {text}

Output:
JSON

{{
  "claim": [The claim made in the social media post, based on both the image text and the caption]
}}

Guidelines:

    The claim should be stated objectively and avoid any subjective language or interpretation.
    If the post makes multiple claims, focus on the most prominent or overarching one but also make sure other claims are also mentioned.
    If the post does not make a clear claim, but rather expresses an opinion or sentiment, rephrase it as a statement about the author's perspective.
    Ensure the claim is grammatically correct and free of spelling errors.
    The claims should be written formally and professionally.
"""
  response: ChatResponse = chat(model='deepseek-r1:14b', messages=[
  {
    'role': 'user',
    'content': cur_claim_prompt,
  },
], format=ClaimGeneratorResponse.model_json_schema(),
options={"temperature": 0.4})
  claim = ClaimGeneratorResponse.model_validate_json(response.message.content)
  post_id_generated_claim[post_id] = claim.claim 
  print(f"{i+1} out of {len(crosslingual_prediction_post_id)}")
  i+= 1
  if i % 25 == 0:
    with open("claims.json", "w") as f:
      json.dump(post_id_generated_claim, f)
    

# Write the dictionary to a JSON file
with open("claims.json", "w") as f:
    json.dump(post_id_generated_claim, f)

520 out of 4000
521 out of 4000
522 out of 4000
523 out of 4000
524 out of 4000
525 out of 4000
526 out of 4000
527 out of 4000
528 out of 4000
529 out of 4000
530 out of 4000
531 out of 4000
532 out of 4000
533 out of 4000
534 out of 4000
535 out of 4000
536 out of 4000
537 out of 4000
538 out of 4000
539 out of 4000
540 out of 4000
541 out of 4000
542 out of 4000
543 out of 4000
544 out of 4000
545 out of 4000
546 out of 4000
547 out of 4000
548 out of 4000
549 out of 4000
550 out of 4000
551 out of 4000
552 out of 4000
553 out of 4000
554 out of 4000
555 out of 4000
556 out of 4000
557 out of 4000
558 out of 4000
559 out of 4000
560 out of 4000
561 out of 4000
562 out of 4000
563 out of 4000
564 out of 4000
565 out of 4000
566 out of 4000
567 out of 4000
568 out of 4000
569 out of 4000
570 out of 4000
571 out of 4000
572 out of 4000
573 out of 4000
574 out of 4000
575 out of 4000
576 out of 4000
577 out of 4000
578 out of 4000
579 out of 4000
580 out of 4000
581 out of 4000
582 out 

1031 out of 4000
1032 out of 4000
1033 out of 4000
1034 out of 4000
1035 out of 4000
1036 out of 4000
1037 out of 4000
1038 out of 4000
1039 out of 4000
1040 out of 4000
1041 out of 4000
1042 out of 4000
1043 out of 4000
1044 out of 4000
1045 out of 4000
1046 out of 4000
1047 out of 4000
1048 out of 4000
1049 out of 4000
1050 out of 4000
1051 out of 4000
1052 out of 4000
1053 out of 4000
1054 out of 4000
1055 out of 4000
1056 out of 4000
1057 out of 4000
1058 out of 4000
1059 out of 4000
1060 out of 4000
1061 out of 4000
1062 out of 4000
1063 out of 4000
1064 out of 4000
1065 out of 4000
1066 out of 4000
1067 out of 4000
1068 out of 4000
1069 out of 4000
1070 out of 4000
1071 out of 4000
1072 out of 4000
1073 out of 4000
1074 out of 4000
1075 out of 4000
1076 out of 4000
1077 out of 4000
1078 out of 4000
1079 out of 4000
1080 out of 4000
1081 out of 4000
1082 out of 4000
1083 out of 4000
1084 out of 4000
1085 out of 4000
1086 out of 4000
1087 out of 4000
1088 out of 4000
1089 out of 40

1513 out of 4000
1514 out of 4000
1515 out of 4000
1516 out of 4000
1517 out of 4000
1518 out of 4000
1519 out of 4000
1520 out of 4000
1521 out of 4000
1522 out of 4000
1523 out of 4000
1524 out of 4000
1525 out of 4000
1526 out of 4000
1527 out of 4000
1528 out of 4000
1529 out of 4000
1530 out of 4000
1531 out of 4000
1532 out of 4000
1533 out of 4000
1534 out of 4000
1535 out of 4000
1536 out of 4000
1537 out of 4000
1538 out of 4000
1539 out of 4000
1540 out of 4000
1541 out of 4000
1542 out of 4000
1543 out of 4000
1544 out of 4000
1545 out of 4000
1546 out of 4000
1547 out of 4000
1548 out of 4000
1549 out of 4000
1550 out of 4000
1551 out of 4000
1552 out of 4000
1553 out of 4000
1554 out of 4000
1555 out of 4000
1556 out of 4000
1557 out of 4000
1558 out of 4000
1559 out of 4000
1560 out of 4000
1561 out of 4000
1562 out of 4000
1563 out of 4000
1564 out of 4000
1565 out of 4000
1566 out of 4000
1567 out of 4000
1568 out of 4000
1569 out of 4000
1570 out of 4000
1571 out of 40

1995 out of 4000
1996 out of 4000
1997 out of 4000
1998 out of 4000
1999 out of 4000
2000 out of 4000
2001 out of 4000
2002 out of 4000
2003 out of 4000
2004 out of 4000
2005 out of 4000
2006 out of 4000
2007 out of 4000
2008 out of 4000
2009 out of 4000
2010 out of 4000
2011 out of 4000
2012 out of 4000
2013 out of 4000
2014 out of 4000
2015 out of 4000
2016 out of 4000
2017 out of 4000
2018 out of 4000
2019 out of 4000
2020 out of 4000
2021 out of 4000
2022 out of 4000
2023 out of 4000
2024 out of 4000
2025 out of 4000
2026 out of 4000
2027 out of 4000
2028 out of 4000
2029 out of 4000
2030 out of 4000
2031 out of 4000
2032 out of 4000
2033 out of 4000
2034 out of 4000
2035 out of 4000
2036 out of 4000
2037 out of 4000
2038 out of 4000
2039 out of 4000
2040 out of 4000
2041 out of 4000
2042 out of 4000
2043 out of 4000
2044 out of 4000
2045 out of 4000
2046 out of 4000
2047 out of 4000
2048 out of 4000
2049 out of 4000
2050 out of 4000
2051 out of 4000
2052 out of 4000
2053 out of 40

2477 out of 4000
2478 out of 4000
2479 out of 4000
2480 out of 4000
2481 out of 4000
2482 out of 4000
2483 out of 4000
2484 out of 4000
2485 out of 4000
2486 out of 4000
2487 out of 4000
2488 out of 4000
2489 out of 4000
2490 out of 4000
2491 out of 4000
2492 out of 4000
2493 out of 4000
2494 out of 4000
2495 out of 4000
2496 out of 4000
2497 out of 4000
2498 out of 4000
2499 out of 4000
2500 out of 4000
2501 out of 4000
2502 out of 4000
2503 out of 4000
2504 out of 4000
2505 out of 4000
2506 out of 4000
2507 out of 4000
2508 out of 4000
2509 out of 4000
2510 out of 4000
2511 out of 4000
2512 out of 4000
2513 out of 4000
2514 out of 4000
2515 out of 4000
2516 out of 4000
2517 out of 4000
2518 out of 4000
2519 out of 4000
2520 out of 4000
2521 out of 4000
2522 out of 4000
2523 out of 4000
2524 out of 4000
2525 out of 4000
2526 out of 4000
2527 out of 4000
2528 out of 4000
2529 out of 4000
2530 out of 4000
2531 out of 4000
2532 out of 4000
2533 out of 4000
2534 out of 4000
2535 out of 40

2959 out of 4000
2960 out of 4000
2961 out of 4000
2962 out of 4000
2963 out of 4000
2964 out of 4000
2965 out of 4000
2966 out of 4000
2967 out of 4000
2968 out of 4000
2969 out of 4000
2970 out of 4000
2971 out of 4000
2972 out of 4000
2973 out of 4000
2974 out of 4000
2975 out of 4000
2976 out of 4000
2977 out of 4000
2978 out of 4000
2979 out of 4000
2980 out of 4000
2981 out of 4000
2982 out of 4000
2983 out of 4000
2984 out of 4000
2985 out of 4000
2986 out of 4000
2987 out of 4000
2988 out of 4000
2989 out of 4000
2990 out of 4000
2991 out of 4000
2992 out of 4000
2993 out of 4000
2994 out of 4000
2995 out of 4000
2996 out of 4000
2997 out of 4000
2998 out of 4000
2999 out of 4000
3000 out of 4000
3001 out of 4000
3002 out of 4000
3003 out of 4000
3004 out of 4000
3005 out of 4000
3006 out of 4000
3007 out of 4000
3008 out of 4000
3009 out of 4000
3010 out of 4000
3011 out of 4000
3012 out of 4000
3013 out of 4000
3014 out of 4000
3015 out of 4000
3016 out of 4000
3017 out of 40

3441 out of 4000
3442 out of 4000
3443 out of 4000
3444 out of 4000
3445 out of 4000
3446 out of 4000
3447 out of 4000
3448 out of 4000
3449 out of 4000
3450 out of 4000
3451 out of 4000
3452 out of 4000
3453 out of 4000
3454 out of 4000
3455 out of 4000
3456 out of 4000
3457 out of 4000
3458 out of 4000
3459 out of 4000
3460 out of 4000
3461 out of 4000
3462 out of 4000
3463 out of 4000
3464 out of 4000
3465 out of 4000
3466 out of 4000
3467 out of 4000
3468 out of 4000
3469 out of 4000
3470 out of 4000
3471 out of 4000
3472 out of 4000
3473 out of 4000
3474 out of 4000
3475 out of 4000
3476 out of 4000
3477 out of 4000
3478 out of 4000
3479 out of 4000
3480 out of 4000
3481 out of 4000
3482 out of 4000
3483 out of 4000
3484 out of 4000
3485 out of 4000
3486 out of 4000
3487 out of 4000
3488 out of 4000
3489 out of 4000
3490 out of 4000
3491 out of 4000
3492 out of 4000
3493 out of 4000
3494 out of 4000
3495 out of 4000
3496 out of 4000
3497 out of 4000
3498 out of 4000
3499 out of 40

3923 out of 4000
3924 out of 4000
3925 out of 4000
3926 out of 4000
3927 out of 4000
3928 out of 4000
3929 out of 4000
3930 out of 4000
3931 out of 4000
3932 out of 4000
3933 out of 4000
3934 out of 4000
3935 out of 4000
3936 out of 4000
3937 out of 4000
3938 out of 4000
3939 out of 4000
3940 out of 4000
3941 out of 4000
3942 out of 4000
3943 out of 4000
3944 out of 4000
3945 out of 4000
3946 out of 4000
3947 out of 4000
3948 out of 4000
3949 out of 4000
3950 out of 4000
3951 out of 4000
3952 out of 4000
3953 out of 4000
3954 out of 4000
3955 out of 4000
3956 out of 4000
3957 out of 4000
3958 out of 4000
3959 out of 4000
3960 out of 4000
3961 out of 4000
3962 out of 4000
3963 out of 4000
3964 out of 4000
3965 out of 4000
3966 out of 4000
3967 out of 4000
3968 out of 4000
3969 out of 4000
3970 out of 4000
3971 out of 4000
3972 out of 4000
3973 out of 4000
3974 out of 4000
3975 out of 4000
3976 out of 4000
3977 out of 4000
3978 out of 4000
3979 out of 4000
3980 out of 4000
3981 out of 40

In [ ]:
with open(f'claims.json', 'r') as claims_file:
  claims_data = json.load(claims_file)

In [ ]:
class ClaimCompareResponse(BaseModel):
    can_fact_check: bool

In [ ]:
final_ans = {}
with open("final_ans.json", "r") as final_ans_file:
    final_ans = json.load(final_ans_file)


for post_id, claim in zip(list(claims_data.keys()), list(claims_data.values())):     
    print(f"current post id: {post_id}")
    cur_list = []
    
    post_number = list(claims_data.keys()).index(post_id)

    cur_num_vectors_start = 0

    

    while len(cur_list) < 10:
        cur_num_vectors = cur_num_vectors_start+20
        response = collection.query(
        query_texts=[claim],
        n_results=cur_num_vectors
        )
        chroma_ids = [int(id) for id in response["ids"][0]]
        chroma_documents = response["documents"][0]
        cur_fact_check = 1
        
        for fact_check_id, claim_to_check_against in zip(chroma_ids[cur_num_vectors_start:cur_num_vectors], chroma_documents[cur_num_vectors_start:cur_num_vectors]):
            claim_check_prompt = f"""
**Task:**

Determine whether a fact-checked claim (Claim A) can be used to verify or contradict another claim (Claim B).  In other words, does knowing that Claim A is true help us determine the truth value of Claim B?

**Input:**

You will be provided with two claims:

*   **Claim A:**  A statement that has already been fact-checked and is considered TRUE.
*   **Claim B:** A statement that needs to be evaluated in light of Claim A.

**Output:**

Respond in JSON format with the following key-value pairs:

```json
{{
  "can_fact_check": boolean 
  
}}
    "can_fact_check": true if Claim A can be used to verify or condratict Claim B; false otherwise. This should be based on the generated explanation.
    
Detailed Guidance:

    Entity and Concept Extraction:
        Identify all significant entities in each claim: people, locations, organizations, dates, specific objects, etc.
        Identify the central concept or relationship being asserted in each claim (e.g., cause-and-effect, possession, location, action, attribute).

    Comparative Analysis:
        Overlap: Does Claim B mention the same entities or concepts as Claim A, or closely related ones (synonyms, different aspects of the same entity)?
        Contradiction: Does Claim B directly contradict any part of Claim A? If Claim A is true, does that make Claim B impossible or highly unlikely?
        Support: Does Claim A provide direct evidence or strong supporting information for Claim B? If Claim A is true, does that increase the likelihood of Claim B being true?
        Irrelevance: Are the entities, concepts, and central themes of Claim A and Claim B completely unrelated? If so, Claim A likely cannot be used to fact-check Claim B.

    Logical Inference:
        Consider whether the truth of Claim A logically implies the truth or falsehood of Claim B.
        Even if claims are not identical, if they discuss the same general topic or event, and Claim A provides information relevant to the truth of Claim B, then "can_fact_check" should be true.
        Remember that Claim A is assumed to be TRUE. Your task is to assess whether this true statement has any bearing on Claim B.

    Central Theme Summarization
        Concisely capture the essential meaning of each claim in a single, clear sentence. Focus on the core subject and what is being asserted about it.
        
Here is your input for the Task:

    Claim A: {claim_to_check_against}
    Claim B: {claim}


"""
            response: ChatResponse = client.chat(model='deepseek-r1:7b', messages=[
      {
        'role': 'user',
        'content': claim_check_prompt,
      },
    ], 
    format=ClaimCompareResponse.model_json_schema(),
    options={"temperature": 0.4}
    )
            claim_check_response = ClaimCompareResponse.model_validate_json(response.message.content)
        
            if claim_check_response.can_fact_check:
                cur_list.append(fact_check_id)
                final_ans[post_id] = cur_list
                with open("final_ans.json", "w") as f1:
                    json.dump(final_ans, f1)
            
                if len(cur_list) >= 10:
                    break
            print(f"post number: {post_number}, iteration: {cur_fact_check}")
        
            if cur_fact_check  % 25 == 0:
                final_ans[post_id] = cur_list
                with open("not_inferred_2.json", "w") as f1:
                    json.dump(final_ans, f1)
                
            cur_fact_check += 1
        cur_num_vector_start = cur
    
         post_number += 1

         final_ans[post_id] = cur_list
    

        

        
       
# Write the dictionary to a JSON file
with open("final_ans.json", "w") as f1:
    json.dump(final_ans, f1)

same claim! =========================
post number: 420, iteration: 1
same claim! =========================
post number: 420, iteration: 2
post number: 420, iteration: 3
post number: 420, iteration: 4
post number: 420, iteration: 5
post number: 420, iteration: 6
same claim! =========================
post number: 420, iteration: 7
post number: 420, iteration: 8
post number: 420, iteration: 9
post number: 420, iteration: 10
post number: 420, iteration: 11
post number: 420, iteration: 12
post number: 420, iteration: 13
post number: 420, iteration: 14
post number: 420, iteration: 15
post number: 420, iteration: 16
post number: 420, iteration: 17
post number: 420, iteration: 18
post number: 420, iteration: 19
post number: 420, iteration: 20
current post id: 74861
same claim! =========================
post number: 421, iteration: 1
same claim! =========================
post number: 421, iteration: 2
post number: 421, iteration: 3
post number: 421, iteration: 4
post number: 421, iteration: 5
p

post number: 429, iteration: 16
post number: 429, iteration: 17
post number: 429, iteration: 18
post number: 429, iteration: 19
post number: 429, iteration: 20
current post id: 74934
same claim! =========================
post number: 430, iteration: 1
post number: 430, iteration: 2
post number: 430, iteration: 3
post number: 430, iteration: 4
post number: 430, iteration: 5
post number: 430, iteration: 6
post number: 430, iteration: 7
post number: 430, iteration: 8
post number: 430, iteration: 9
same claim! =========================
post number: 430, iteration: 10
post number: 430, iteration: 11
post number: 430, iteration: 12
post number: 430, iteration: 13
same claim! =========================
post number: 430, iteration: 14
post number: 430, iteration: 15
post number: 430, iteration: 16
post number: 430, iteration: 17
post number: 430, iteration: 18
post number: 430, iteration: 19
post number: 430, iteration: 20
current post id: 74989
post number: 431, iteration: 1
same claim! ======

post number: 439, iteration: 15
same claim! =========================
post number: 439, iteration: 16
same claim! =========================
10 fact checks for post: 75135 =========================
current post id: 75144
post number: 440, iteration: 1
post number: 440, iteration: 2
post number: 440, iteration: 3
same claim! =========================
post number: 440, iteration: 4
post number: 440, iteration: 5
post number: 440, iteration: 6
post number: 440, iteration: 7
same claim! =========================
post number: 440, iteration: 8
post number: 440, iteration: 9
post number: 440, iteration: 10
same claim! =========================
post number: 440, iteration: 11
post number: 440, iteration: 12
post number: 440, iteration: 13
post number: 440, iteration: 14
post number: 440, iteration: 15
post number: 440, iteration: 16
post number: 440, iteration: 17
same claim! =========================
post number: 440, iteration: 18
post number: 440, iteration: 19
post number: 440, iteration: 

same claim! =========================
post number: 450, iteration: 2
post number: 450, iteration: 3
same claim! =========================
post number: 450, iteration: 4
same claim! =========================
post number: 450, iteration: 5
same claim! =========================
post number: 450, iteration: 6
same claim! =========================
post number: 450, iteration: 7
same claim! =========================
post number: 450, iteration: 8
post number: 450, iteration: 9
post number: 450, iteration: 10
post number: 450, iteration: 11
post number: 450, iteration: 12
post number: 450, iteration: 13
post number: 450, iteration: 14
same claim! =========================
post number: 450, iteration: 15
post number: 450, iteration: 16
post number: 450, iteration: 17
post number: 450, iteration: 18
post number: 450, iteration: 19
post number: 450, iteration: 20
current post id: 75284
same claim! =========================
post number: 451, iteration: 1
post number: 451, iteration: 2
post number

same claim! =========================
post number: 460, iteration: 2
post number: 460, iteration: 3
post number: 460, iteration: 4
post number: 460, iteration: 5
same claim! =========================
post number: 460, iteration: 6
same claim! =========================
post number: 460, iteration: 7
post number: 460, iteration: 8
post number: 460, iteration: 9
post number: 460, iteration: 10
same claim! =========================
post number: 460, iteration: 11
post number: 460, iteration: 12
same claim! =========================
post number: 460, iteration: 13
post number: 460, iteration: 14
post number: 460, iteration: 15
same claim! =========================
post number: 460, iteration: 16
post number: 460, iteration: 17
post number: 460, iteration: 18
post number: 460, iteration: 19
post number: 460, iteration: 20
current post id: 75496
same claim! =========================
post number: 461, iteration: 1
same claim! =========================
post number: 461, iteration: 2
same claim!

...

post number: 560, iteration: 12
post number: 560, iteration: 13
post number: 560, iteration: 14
post number: 560, iteration: 15
post number: 560, iteration: 16
post number: 560, iteration: 17
post number: 560, iteration: 18
post number: 560, iteration: 19
post number: 560, iteration: 20
current post id: 77032
same claim! =========================
post number: 561, iteration: 1
same claim! =========================
post number: 561, iteration: 2
post number: 561, iteration: 3
post number: 561, iteration: 4
post number: 561, iteration: 5
post number: 561, iteration: 6
post number: 561, iteration: 7
post number: 561, iteration: 8
post number: 561, iteration: 9
post number: 561, iteration: 10
post number: 561, iteration: 11
post number: 561, iteration: 12
post number: 561, iteration: 13
post number: 561, iteration: 14
post number: 561, iteration: 15
post number: 561, iteration: 16
post number: 561, iteration: 17
post number: 561, iteration: 18
post number: 561, iteration: 19
post number: 5

post number: 570, iteration: 11
post number: 570, iteration: 12
post number: 570, iteration: 13
same claim! =========================
post number: 570, iteration: 14
post number: 570, iteration: 15
post number: 570, iteration: 16
post number: 570, iteration: 17
post number: 570, iteration: 18
same claim! =========================
post number: 570, iteration: 19
same claim! =========================
post number: 570, iteration: 20
current post id: 77172
same claim! =========================
post number: 571, iteration: 1
same claim! =========================
post number: 571, iteration: 2
same claim! =========================
post number: 571, iteration: 3
same claim! =========================
post number: 571, iteration: 4
same claim! =========================
post number: 571, iteration: 5
post number: 571, iteration: 6
same claim! =========================
post number: 571, iteration: 7
same claim! =========================
post number: 571, iteration: 8
same claim! =================

post number: 579, iteration: 17
post number: 579, iteration: 18
post number: 579, iteration: 19
post number: 579, iteration: 20
current post id: 77411
same claim! =========================
post number: 580, iteration: 1
same claim! =========================
post number: 580, iteration: 2
same claim! =========================
post number: 580, iteration: 3
post number: 580, iteration: 4
post number: 580, iteration: 5
post number: 580, iteration: 6
post number: 580, iteration: 7
post number: 580, iteration: 8
same claim! =========================
post number: 580, iteration: 9
post number: 580, iteration: 10
post number: 580, iteration: 11
post number: 580, iteration: 12
post number: 580, iteration: 13
same claim! =========================
post number: 580, iteration: 14
same claim! =========================
post number: 580, iteration: 15
post number: 580, iteration: 16
same claim! =========================
post number: 580, iteration: 17
post number: 580, iteration: 18
post number: 580

same claim! =========================
post number: 589, iteration: 1
post number: 589, iteration: 2
same claim! =========================
post number: 589, iteration: 3
post number: 589, iteration: 4
post number: 589, iteration: 5
post number: 589, iteration: 6
post number: 589, iteration: 7
same claim! =========================
post number: 589, iteration: 8
post number: 589, iteration: 9
same claim! =========================
post number: 589, iteration: 10
post number: 589, iteration: 11
same claim! =========================
post number: 589, iteration: 12
same claim! =========================
post number: 589, iteration: 13
post number: 589, iteration: 14
post number: 589, iteration: 15
post number: 589, iteration: 16
same claim! =========================
post number: 589, iteration: 17
post number: 589, iteration: 18
same claim! =========================
post number: 589, iteration: 19
same claim! =========================
post number: 589, iteration: 20
current post id: 77584
same

same claim! =========================
post number: 598, iteration: 1
same claim! =========================
post number: 598, iteration: 2
same claim! =========================
post number: 598, iteration: 3
post number: 598, iteration: 4
same claim! =========================
post number: 598, iteration: 5
same claim! =========================
post number: 598, iteration: 6
same claim! =========================
post number: 598, iteration: 7
post number: 598, iteration: 8
same claim! =========================
post number: 598, iteration: 9
same claim! =========================
post number: 598, iteration: 10
same claim! =========================
post number: 598, iteration: 11
same claim! =========================
10 fact checks for post: 77852 =========================
current post id: 77903
post number: 599, iteration: 1
post number: 599, iteration: 2
post number: 599, iteration: 3
post number: 599, iteration: 4
post number: 599, iteration: 5
post number: 599, iteration: 6
post number